In [6]:
import os
import sys
sys.path.append("./ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.set_default_dtype(torch.float32)

from ddn.ddn.pytorch.node import AbstractDeclarativeNode
# from utils.bernstein import bernstein_coeff_order10_new

In [7]:
import torch
from scipy.special import binom

# Reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.set_default_dtype(torch.float64)

def bernstein_coeff_order10_new(n, tmin, tmax, t_actual):
    
    l = tmax-tmin
    t = (t_actual-tmin)/l
    
    P0 = binom(n,0)*((1-t)**(n-0))*t**0
    P1 = binom(n,1)*((1-t)**(n-1))*t**1                     
    P2 = binom(n,2)*((1-t)**(n-2))*t**2
    P3 = binom(n,3)*((1-t)**(n-3))*t**3                        
    P4 = binom(n,4)*((1-t)**(n-4))*t**4                    
    P5 = binom(n,5)*((1-t)**(n-5))*t**5
    P6 = binom(n,6)*((1-t)**(n-6))*t**6
    P7 = binom(n,7)*((1-t)**(n-7))*t**7
    P8 = binom(n,8)*((1-t)**(n-8))*t**8
    P9 = binom(n,9)*((1-t)**(n-9))*t**9
    P10 = binom(n,10)*((1-t)**(n-10))*t**10
    
    P0dot = -10.0*(-t + 1)**9
    P1dot = -90.0*t*(-t + 1)**8 + 10.0*(-t + 1)**9
    P2dot = -360.0*t**2*(-t + 1)**7 + 90.0*t*(-t + 1)**8
    P3dot = -840.0*t**3*(-t + 1)**6 + 360.0*t**2*(-t + 1)**7
    P4dot = -1260.0*t**4*(-t + 1)**5 + 840.0*t**3*(-t + 1)**6
    P5dot = -1260.0*t**5*(-t + 1)**4 + 1260.0*t**4*(-t + 1)**5
    P6dot = -840.0*t**6*(-t + 1)**3 + 1260.0*t**5*(-t + 1)**4
    P7dot = -360.0*t**7*(-t + 1)**2 + 840.0*t**6*(-t + 1)**3
    P8dot = 45.0*t**8*(2*t - 2) + 360.0*t**7*(-t + 1)**2
    P9dot = -10.0*t**9 + 9*t**8*(-10.0*t + 10.0)
    P10dot = 10.0*t**9
    
    P0ddot = 90.0*(-t + 1)**8
    P1ddot = 720.0*t*(-t + 1)**7 - 180.0*(-t + 1)**8
    P2ddot = 2520.0*t**2*(-t + 1)**6 - 1440.0*t*(-t + 1)**7 + 90.0*(-t + 1)**8
    P3ddot = 5040.0*t**3*(-t + 1)**5 - 5040.0*t**2*(-t + 1)**6 + 720.0*t*(-t + 1)**7
    P4ddot = 6300.0*t**4*(-t + 1)**4 - 10080.0*t**3*(-t + 1)**5 + 2520.0*t**2*(-t + 1)**6
    P5ddot = 5040.0*t**5*(-t + 1)**3 - 12600.0*t**4*(-t + 1)**4 + 5040.0*t**3*(-t + 1)**5
    P6ddot = 2520.0*t**6*(-t + 1)**2 - 10080.0*t**5*(-t + 1)**3 + 6300.0*t**4*(-t + 1)**4
    P7ddot = -360.0*t**7*(2*t - 2) - 5040.0*t**6*(-t + 1)**2 + 5040.0*t**5*(-t + 1)**3
    P8ddot = 90.0*t**8 + 720.0*t**7*(2*t - 2) + 2520.0*t**6*(-t + 1)**2
    P9ddot = -180.0*t**8 + 72*t**7*(-10.0*t + 10.0)
    P10ddot = 90.0*t**8
    
    P = torch.hstack((P0, P1, P2, P3, P4, P5, P6, P7, P8, P9, P10 ))    
    Pdot = torch.hstack((P0dot, P1dot, P2dot, P3dot, P4dot, P5dot, P6dot, P7dot, P8dot, P9dot, P10dot ))/l
    Pddot = torch.hstack((P0ddot, P1ddot, P2ddot, P3ddot, P4ddot, P5ddot, P6ddot, P7ddot, P8ddot, P9ddot, P10ddot ))/(l**2)
    
    return P, Pdot, Pddot

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [34]:
# DDN QP
class QPNode(AbstractDeclarativeNode):
    def __init__(self, weight_smoothness, t_fin, num, num_batch, device=device):
        super().__init__(eps=1e-6, gamma=6e-3, chunk_size=None)
        
        # Initialization
        self.weight_smoothness = weight_smoothness
        self.num = num
        self.t_fin = t_fin
        self.t = self.t_fin / self.num
        self.num_batch = num_batch  # number of goals
        
        tot_time = torch.linspace(0, t_fin, num, device=device)
        tot_time_copy = tot_time.reshape(num, 1)
        
        self.tot_time = tot_time
        
        self.P, self.Pdot, self.Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
        print(self.P.shape)
        self.P_torch, self.Pdot_torch, self.Pddot_torch = self.P.to(device), self.Pdot.to(device), self.Pddot.to(device)
        
        self.nvar = self.P_torch.shape[1] - 3
        
        self.P_offset = self.P_torch[:, 0:3]
        self.Pdot_offset = self.Pdot_torch[:, 0:3]
        self.Pddot_offset = self.Pddot_torch[:, 0:3]
        
        self.P_bar = self.P_torch[:, 3:]
        self.Pdot_bar = self.Pdot_torch[:, 3:]
        self.Pddot_bar = self.Pddot_torch[:, 3:]

        self.A_goal = torch.vstack([self.P_bar[-1], self.Pdot_bar[-1], self.Pddot_bar[-1]])
        print(self.A_goal.shape)
        self.Q_smoothness = self.weight_smoothness * torch.mm(self.Pddot_bar.T, self.Pddot_bar)
        self.Q_goal = torch.mm(self.A_goal.T, self.A_goal)

    def objective(self, b_goal_state, y):
        
        c_x = y[:, 0:self.nvar]
        c_y = y[:, self.nvar:2 * self.nvar]
                
        x_f, y_f, vx_f, vy_f, ax_f, ay_f = [b_goal_state[:, i] for i in range(b_goal_state.shape[1])]
        
        x = torch.mm(self.P_bar, c_x.T).T + self.x_offset
        vx = torch.mm(self.Pdot_bar, c_x.T).T + self.vx_offset
        ax = torch.mm(self.Pddot_bar, c_x.T).T + self.ax_offset

        y = torch.mm(self.P_bar, c_y.T).T + self.y_offset
        vy = torch.mm(self.Pdot_bar, c_y.T).T + self.vy_offset
        ay = torch.mm(self.Pddot_bar, c_y.T).T + self.ay_offset
        
        # Cost function
        cost = 0.5 * self.weight_smoothness * (torch.sum(ax ** 2, 1) + torch.sum(ay ** 2 , 1)) + \
               0.5 * ( (x[:, -1] - x_f) ** 2 + (vx[:, -1] - vx_f) ** 2 + (ax[:, -1] - ax_f) ** 2 + \
                       (y[:, -1] - y_f) ** 2 + (vy[:, -1] - vy_f) ** 2 + (ay[:, -1] - ay_f) ** 2)

        return cost

    def solve(self, fixed_params, b_pred):
                        
        x_init, y_init, vx_init, vy_init, ax_init, ay_init = [fixed_params[:, i] for i in range(fixed_params.shape[1])]
        x_f, y_f, vx_f, vy_f, ax_f, ay_f = [b_pred[:, i] for i in range(b_pred.shape[1])]
                
        P_00 = self.P_torch[0, 0]
        P_01 = self.P_torch[0, 1]
        P_02 = self.P_torch[0, 2]

        Pdot_00 = self.Pdot_torch[0, 0]
        Pdot_01 = self.Pdot_torch[0, 1]
        Pdot_02 = self.Pdot_torch[0, 2]

        Pddot_00 = self.Pddot_torch[0, 0]
        Pddot_01 = self.Pddot_torch[0, 1]
        Pddot_02 = self.Pddot_torch[0, 2]

        cx_1 = x_init
        cx_2 = (vx_init - cx_1 * Pdot_00) / Pdot_01
        cx_3 = (ax_init - cx_1 * Pddot_00 - cx_2 * Pddot_01) / Pddot_02

        cy_1 = y_init
        cy_2 = (vy_init - cy_1 * Pdot_00) / Pdot_01
        cy_3 = (ay_init - cy_1 * Pddot_00 - cy_2 * Pddot_01) / Pddot_02

        cx_offset = torch.vstack((cx_1, cx_2, cx_3)).T
        cy_offset = torch.vstack((cy_1, cy_2, cy_3)).T

        self.x_offset = torch.mm(self.P_offset, cx_offset.T ).T
        self.vx_offset  = torch.mm(self.Pdot_offset, cx_offset.T ).T
        self.ax_offset = torch.mm(self.Pddot_offset, cx_offset.T ).T

        self.y_offset = torch.mm(self.P_offset, cy_offset.T ).T
        self.vy_offset  = torch.mm(self.Pdot_offset, cy_offset.T ).T
        self.ay_offset = torch.mm(self.Pddot_offset, cy_offset.T ).T

        b_x_goal = (- self.x_offset[:, -1] + x_f).reshape(self.num_batch, 1)
        b_y_goal = (- self.y_offset[:, -1] + y_f).reshape(self.num_batch, 1)

        b_vx_goal = (- self.vx_offset[:, -1] + vx_f).reshape(self.num_batch, 1)
        b_vy_goal = (- self.vy_offset[:, -1] + vy_f).reshape(self.num_batch, 1)

        b_ax_goal = (- self.ax_offset[:, -1] + ax_f).reshape(self.num_batch, 1)
        b_ay_goal = (- self.ay_offset[:, -1] + ay_f).reshape(self.num_batch, 1)

        b_goal_state_x = torch.hstack((b_x_goal, b_vx_goal, b_ax_goal))
        b_goal_state_y = torch.hstack((b_y_goal, b_vy_goal, b_ay_goal))

        q_x_smoothness = torch.mm(self.Pddot_bar.T, self.ax_offset.T).T 
        q_x_goal = - torch.mm(self.A_goal.T, b_goal_state_x.T).T 
        cost_x = self.Q_smoothness + self.Q_goal
        lincost_x = q_x_smoothness + q_x_goal
        sol_x = torch.linalg.solve(- cost_x, lincost_x.T).T
    
        q_y_smoothness = torch.mm(self.Pddot_bar.T, self.ay_offset.T).T 
        q_y_goal = - torch.mm(self.A_goal.T, b_goal_state_y.T).T 
        cost_y = self.Q_smoothness + self.Q_goal
        lincost_y = q_y_smoothness + q_y_goal        
        sol_y = torch.linalg.solve(- cost_y, lincost_y.T).T
                
        sol = torch.hstack([sol_x, sol_y])
        
        offset = torch.hstack([self.x_offset, self.y_offset])
        
        return sol, None, offset

In [35]:
class QPFunction(torch.autograd.Function):
    """Generic declarative autograd function.
    Defines the forward and backward functions. Saves all inputs and outputs,
    which may be memory-inefficient for the specific problem.
    
    Assumptions:
    * All inputs are PyTorch tensors
    * All inputs have a single batch dimension (b, ...)
    """
    @staticmethod
    def forward(ctx, problem, *inputs):
        output, solve_ctx, offset = torch.no_grad()(problem.solve)(*inputs)
        ctx.save_for_backward(output, inputs[-1])
        ctx.problem = problem
        ctx.solve_ctx = solve_ctx
        return output.clone(), offset.clone()

    @staticmethod
    def backward(ctx, grad_output, offset):
        output, inputs = ctx.saved_tensors
        problem = ctx.problem
        solve_ctx = ctx.solve_ctx
        output.requires_grad = True
        offset.requires_grad = False
        # inputs = tuple(inputs)
        grad_inputs = problem.gradient(inputs, y=output, v=grad_output,
            ctx=solve_ctx)
        return (None, None, *grad_inputs)
    
class DeclarativeLayer(torch.nn.Module):
    """Generic declarative layer.
    
    Assumptions:
    * All inputs are PyTorch tensors
    * All inputs have a single batch dimension (b, ...)
    Usage:
        problem = <derived class of *DeclarativeNode>
        declarative_layer = DeclarativeLayer(problem)
        y = declarative_layer(x1, x2, ...)
    """
    def __init__(self, problem):
        super(DeclarativeLayer, self).__init__()
        self.problem = problem
        
    def forward(self, *inputs):
        return QPFunction.apply(self.problem, *inputs)

In [36]:
# Prevents NaN by torch.log(0)
def torch_log(x):
    return torch.log(torch.clamp(x, min = 1e-10))

# Encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, z_dim):
        super(Encoder, self).__init__()
        
        # GRU
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size=input_size + output_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        
        # Regularization
        self.bn = nn.BatchNorm1d(num_features=hidden_size)
        self.dropout = nn.Dropout(p=0.25)
        
        # Feed Forward Layer
        self.linear1 = nn.Linear(hidden_size, 64)
        self.activation = nn.ReLU()
        
        # Mean and Variance
        self.mu = nn.Linear(64, z_dim)
        self.var = nn.Linear(64, z_dim)
        
        self.softplus = nn.Softplus()
        
    def forward(self, x):
                
        batch_size = x.shape[0]
        h0 = torch.zeros((self.num_layers, batch_size, self.hidden_size), device=device, requires_grad=True)
        
        out, hn = self.gru(x.view(batch_size, 1, -1), (h0))
        
        out = self.activation(self.linear1(self.bn(hn[0])))
        out = self.dropout(out)
        
        mu = self.mu(out)
        var = self.var(out)
        
        return mu, self.softplus(var)
    
# Decoder
class Decoder(nn.Module):
    def __init__(self, opt_layer, P, input_size, hidden_size, output_size, num_layers, z_dim):
        super(Decoder, self).__init__()
        
        # GRU
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size=z_dim + input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        
        # Regularization
        self.bn = nn.BatchNorm1d(num_features=hidden_size)
        self.dropout = nn.Dropout(p=0.25)
        
        # Feed Forward Layer
        self.linear1 = nn.Linear(hidden_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)
        self.activation = nn.ReLU()
        
        # Optimization
        self.P = P
        self.opt_layer = opt_layer
    
    def forward(self, x, inputs):
        batch_size = x.shape[0]
        h0 = torch.zeros((self.num_layers, batch_size, self.hidden_size), device=device, requires_grad=True)
        out, hn = self.gru(x.view(batch_size, 1, -1), (h0))
        
        out = self.activation(self.linear1(self.bn(hn[0])))
        out = self.dropout(out)
        
        b_pred = self.linear2(out)
        
        # Run Optimization
        sol, offset = self.opt_layer(inputs, b_pred)
        
        print(sol.shape, offset.shape)
        # Compute final trajectory
        x_traj_batch = torch.mm(self.P, sol[:, :8].T).T + offset[:, 0:12]
        y_traj_batch = torch.mm(self.P, sol[:, 8:].T).T + offset[:, 12:]
        traj = torch.hstack([x_traj_batch, y_traj_batch])
        
        return traj

class GRU_cVAE(nn.Module):
    def __init__(self, encoder, decoder):
        super(GRU_cVAE, self).__init__()
        
        # Encoder & Decoder
        self.encoder = encoder
        self.decoder = decoder
        
        # RCL Loss
        self.rcl_loss = nn.MSELoss()
        self.softplus = nn.Softplus()
        
    # Encode Past & Future Trajectories
    def _encoder(self, past, future):
        inp = torch.cat([past, future], dim = 1)
        mean, std = self.encoder(inp)        
        return mean, std

    # Reparametrization Trick
    def _sample_z(self, mean, std):
        eps = torch.randn_like(mean).to(device)
        return mean + std * eps

    # Reconstruct future trajectories given z & past trajectories + initial conditions
    def _decoder(self, z, past, b_inp):
        inp = torch.cat([z, past], dim = 1)
        y = self.decoder(inp, b_inp)
        return y
    
    # Forward Pass
    def forward(self, past, future, b_inp, beta = 1.0, step = 0):
        
        # Beta Annealing
        beta_d = min(step / 1000 * beta, beta)
        
        # Mu & Variance
        mean, std = self._encoder(past, future)
        
        # Sample from z -> Reparameterized 
        z = self._sample_z(mean, std)
        
        # Decode y
        y = self._decoder(z, past, b_inp)
        
        # KL Loss
        KL = -0.5 * torch.mean(torch.sum(1 + torch_log(std ** 2) - mean ** 2 - std ** 2, dim=1))
        
        # RCL Loss 
        RCL = self.rcl_loss(future, y)
        
        # ELBO Loss
        loss = beta_d * KL + RCL
        
        return KL, RCL, loss, beta_d

In [37]:
class TrajectoryDataset(Dataset):
    def __init__(self, data_path, t_obs=8, dt=0.4):

        self.data = np.load(data_path).astype(np.float32)
        self.t_obs = t_obs
        self.dt = dt

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        traj = self.data[idx]
        
        # x - B x 20
        x_traj = traj[:, 0]
        
        # y - B x 20
        y_traj = traj[:, 1]
        
        x_traj = x_traj - x_traj[0]
        y_traj = y_traj - y_traj[0]
        
        # Inputs B x 8
        x_inp = x_traj[:self.t_obs]
        y_inp = y_traj[:self.t_obs]
        
        # Output - B x 12
        x_fut = x_traj[self.t_obs:]
        y_fut = y_traj[self.t_obs:]
    
        # Init Conditions
        vx_beg = (x_inp[-1]-x_inp[-2]) / self.dt
        vy_beg = (y_inp[-1]-y_inp[-2]) / self.dt

        vx_beg_prev = (x_inp[-2]-x_inp[-3]) / self.dt
        vy_beg_prev = (y_inp[-2]-y_inp[-3]) / self.dt

        ax_beg = (vx_beg - vx_beg_prev) / self.dt
        ay_beg = (vy_beg - vy_beg_prev) / self.dt

        traj_inp = np.hstack((x_inp, y_inp)).flatten()
        traj_out = np.hstack((x_fut, y_fut)).flatten()
        b_inp = np.array([x_inp[-1], y_inp[-1], vx_beg, vy_beg, ax_beg, ay_beg])
        
        return torch.tensor(traj_inp).double(), torch.tensor(traj_out).double(), torch.tensor(b_inp).double()
    
def rotate(gt_x, gt_y,theta):
    gt_x_x = [ (gt_x[k] * np.cos(theta) - gt_y[k] * np.sin(theta))  for k in range(len(gt_x))]
    gt_y_y = [ (gt_x[k] * np.sin(theta) + gt_y[k] * np.cos(theta))  for k in range(len(gt_x))]
    gt_x = gt_x_x
    gt_y = gt_y_y
    return gt_x, gt_y

class TrajDataset(Dataset):
    """Expert Trajectory Dataset."""
    def __init__(self, dataset_path):
        
        self.dataset_path = dataset_path
        self.files = os.listdir(DATASET_PATH)
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        arr = np.load(os.path.join(self.dataset_path, self.files[idx], f"{self.files[idx]}.npy"), allow_pickle=True)
        # map_info = np.load(os.path.join(self.dataset_path, self.files[idx], f"lanes_{self.files[idx]}.npy"), allow_pickle=True)
        agent, obstacle_tracks = None, []
        # [object_state.position[0], object_state.position[1], object_state.heading, object_state.velocity[0], object_state.velocity[1], timestamps_ns[ind], tck[track.category], vmp[track.object_type]
        # 5s of input of agent and 4 obstacles, 5s of output
        inp = np.zeros((200))
        out = np.zeros((120))
        for tracks in arr:
            if tracks[0][-2] == 3:
                agent = tracks
                last_obs = np.array(tracks)[-1]
                last_obs_x = last_obs[0]
                last_obs_y = last_obs[1]
                last_obs_vel_x = last_obs[3]
                last_obs_vel_y = last_obs[4]
                for i in range(np.array(tracks).shape[0], 110):
                    timestep = (i - np.array(tracks).shape[0] + 1)
                    tracks.append([last_obs_x + last_obs_vel_x * timestep, last_obs_y + last_obs_vel_y * timestep, last_obs[2], last_obs_vel_x, last_obs_vel_y, i, last_obs[-2], last_obs[-1]])
                agx = np.array(tracks)[10:50, 0] # position-x
                agy = np.array(tracks)[10:50, 1] # position-y
                offsetx = agx[-1]
                offsety = agy[-1]
                theta = np.arctan2(agy[-1] - agy[-2], agx[-1] - agx[-2])
                ox = np.array(tracks)[50:, 0] # position-x
                oy = np.array(tracks)[50:, 1] # position-x
                agx, agy = rotate(agx - offsetx, agy - offsety, -theta)
                ox, oy = rotate(ox - offsetx, oy - offsety, -theta)
                vx = np.array(tracks)[10:50, 3] # velocity-x
                vy = np.array(tracks)[10:50, 4] # velocity-y
                vx, vy = rotate(vx, vy, -theta)
                inp[0::5] = agx
                inp[1::5] = agy
                inp[2::5] = vx
                inp[3::5] = vy
                inp[4::5] = np.array(tracks)[10:50, 2] - theta # heading
                out[:60] = ox
                out[60:] = oy
                traj_inp = np.hstack((agx, agy)).flatten()
                traj_out = np.hstack((ox, oy)).flatten()
                b_inp = np.array([agx[-1], agy[-1], vx[-1], vy[-1], 0, 0])
                
        return torch.tensor(traj_inp).double(), torch.tensor(traj_out).double(), torch.tensor(b_inp).double() #torch.tensor(inp).double(), torch.tensor(out).double()

    
DATASET_PATH =  "/mnt/e/datasets/argoverse/parsed"
import os
files = os.listdir(DATASET_PATH)
    
train_dataset = TrajDataset(DATASET_PATH)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
for _, data in enumerate(train_loader):
    inp, traj_out, b_inp = data
    print(inp.shape, traj_out.shape, b_inp.shape)
    break
    pass
# Using PyTorch Dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)    

torch.Size([64, 80]) torch.Size([64, 120]) torch.Size([64, 6])


In [38]:
# NGSIM - 8 Observed ( 3.2s ) & 12 Pred ( 4.8s )

weight_smoothness = 1.0
t_fin = 6
num_batch = train_loader.batch_size
num = 60

# P_bar
tot_time = torch.linspace(0, t_fin, num, device=device)
tot_time_copy = tot_time.reshape(num, 1)

P, _, _ = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
P_bar = P[:, 3:].to(device) # self.P_torch[:, 3:]

node = QPNode(weight_smoothness, t_fin, num, num_batch, device)
qp_layer = DeclarativeLayer(node)

# cVAE Inputs
input_size = 80
output_size = 120
z_dim = 6
num_layers = 1
hidden_size = 128
decoder_output_size = 6

encoder = Encoder(input_size, hidden_size, output_size, num_layers, z_dim)
decoder = Decoder(qp_layer, P_bar, input_size, hidden_size, decoder_output_size, num_layers, z_dim)
model = GRU_cVAE(encoder, decoder).to(device)

torch.Size([60, 11])
torch.Size([3, 8])


In [30]:
# train_dataset = TrajectoryDataset("./datasets/toy/train_data.npy")
# train_loader = DataLoader(train_dataset, batch_size=num_batch, shuffle=True, num_workers=0, drop_last=True)

# val_dataset = TrajectoryDataset("./datasets/toy/val_data.npy")
# val_loader = DataLoader(val_dataset, batch_size=num_batch, shuffle=True, num_workers=0, drop_last=True)

In [16]:
for batch_num, data in enumerate(train_loader):
    traj_inp, traj_out, b_inp = data
    print(traj_inp.shape, traj_out.shape, b_inp.shape)
    break

torch.Size([64, 80]) torch.Size([64, 120]) torch.Size([64, 6])


In [33]:
epochs = 24
step, beta = 0, 3.5
optimizer = optim.AdamW(model.parameters(), lr = 1e-3, weight_decay=1e-5)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 6, gamma = 0.1)

avg_train_loss, avg_rcl_loss, avg_kl_loss = [], [], []
for epoch in range(epochs):
    
    # Train Loop
    losses_train, kl_losses, rcl_losses = [], [], []
    model.train()
    for past, future, inputs in train_loader:
        
        past = past.to(device)
        future = future.to(device)
        b_inp = inputs.to(device)
                        
        KL_loss, RCL_loss, loss, beta_d = model.forward(past, future, b_inp, beta, step)
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses_train.append(loss.detach().cpu().numpy())
        rcl_losses.append(RCL_loss.detach().cpu().numpy())
        kl_losses.append(KL_loss.detach().cpu().numpy())
    
    print(f"Epoch: {epoch + 1}, Beta: {beta_d}, Step: {step}, Train Loss: {np.average(losses_train):.3f}")
    step += 1.5
    scheduler.step()
    avg_train_loss.append(np.average(losses_train)), avg_rcl_loss.append(np.average(rcl_losses)), avg_kl_loss.append(np.average(kl_losses))

torch.Size([64, 16]) torch.Size([64, 120])


RuntimeError: The size of tensor a (60) must match the size of tensor b (12) at non-singleton dimension 1